In [47]:
import os
import urllib, cStringIO

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import tabular as tb
import pandas as pd
import json
import re

from PIL import Image
import base64

from helpers import *

In [48]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

In [67]:
# directory & file hierarchy
exp_path = '3dObjects/chairs_sketchpad'
analysis_dir = os.getcwd()
data_dir = os.path.abspath(os.path.join(os.getcwd(),'../../..','data',exp_path))
exp_dir = os.path.abspath(os.path.join(os.getcwd(),'../../..','experiments',exp_path))

In [68]:
# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['3dObjects']
coll = db['chairs_sketchpad']

In [89]:
a = coll.find({ '$and': [{'iterationName':'pilot0'}, {'eventType': 'stroke'}, {'workerId': {'$ne':''}}]}).sort('time')
print str(a.count()) + ' strokes in the database.'
a = coll.find({ '$and': [{'iterationName':'pilot0'}, {'eventType': 'clickedObj'}, {'workerId': {'$ne':''}}]}).sort('time')
print str(a.count()) + ' clickedObj trials in the database.'


3080 strokes in the database.
456 clickedObj trials in the database.


In [75]:
all_games = coll.distinct('gameid')
workers = [i[:4] for i in coll.find({'iterationName':'pilot0'}).distinct('workerId') if len(i)>0] 